# <font color=green> Machine Learning: Classificação por trás dos panos
---

# -----------------------------------------NOVOS CONCEITOS DE CLASSIFICAÇÃO-----------------------------------------

    * Como classificar um elemento em 3 categorias
    * Os algoritmos One vs Rest e One vs One
    * A comparação entre os algoritmos OneVsRest, OneVsOne, MultinomialNB e AdaBoostClassifier

- Cliente
    - variáveis:
        * Recencia: O quão recente é o último acesso
        * Frequências: Quantas vezes ele acessou a plataforma
        * Cadastro: Quando ele se cadastrou 
    - Com essas 3 variáveis , quero dizer se meu cliente está: Alegre(2) / Neutro(1) / Insatisfeito(0) com meu produto! (3 CATEGORIAS) 

- Assim, vamos utilizar o ONE VS REST para classifcação com mais de 2 categorias:
    * 0=>0      1=>1,2  LinearSVC 0 ou do resto (38%, resto 62%) 
    * 0=>0,2    1=>1    LinearSVC 1 ou do resto (44%, resto 56%) 
    * 0=>0,1    2=>2    LinearSVC 2 ou do resto (20%, resto 80%) 

* Também podemos utilizar o ONE VS ONE para classifcação com mais de 2 categorias tabém:
    * se tivermos n categorias, vamos rodar n*n classificadores. Onde temos um a um cada categoria 

# -----------------------------------------UTILIZANDO O K-FOLD-----------------------------------------

    * O processo de treino, teste e validação
    * O funcionamento do processo quando a ordem dos seus dados é alterada
    * A validação cruzada com o algoritmo k-fold
    * Como implementar o k-fold

* Quando estamos alterando a ordem dos dados de treino e teste, podemos quebrar em k pedaços de tal forma que , caso sejam:
    * 2 pedaços:: 1°: primeira metade treina e segunda testa && 2°: primeira metade testa e segunda treina
    * 3 pedaços:: 1°: primeiro pedaço treina e resto testa && segundo pedaço treina e o resto testa && terceiro pedaço treina e o resto testa
*  Ou seja, realizo os testes para CADA UMA DAS PARTIÇÕES 

In [3]:
from typing import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score # já vai fazar o fit, predict e devolver as notas

df = pd.read_csv('clientes.csv')
X_df = df[['recencia','frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values 

porcentagem_de_treino_e_teste = 0.9

tamanho_de_treino_e_teste = int(porcentagem_de_treino_e_teste * len(Y))

treino_dados = X[:tamanho_de_treino_e_teste]
treino_marcacoes = Y[:tamanho_de_treino_e_teste]

validacao_dados = X[tamanho_de_treino_e_teste:]
validacao_marcacoes = Y[tamanho_de_treino_e_teste:]

In [4]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = f'A taxa de acerto de {nome} foi: {taxa_de_acerto}'
    print(msg)
    return(taxa_de_acerto)

def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [5]:
resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

A taxa de acerto de OneVsRest foi: 0.9257142857142858
A taxa de acerto de OneVsOne foi: 1.0
A taxa de acerto de MultinomialNB foi: 0.8366666666666667
A taxa de acerto de AdaBoostClassifier foi: 0.7528571428571429


In [6]:
for r in resultados.items():
    print(r)

(0.9257142857142858, OneVsRestClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(1.0, OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(0.8366666666666667, MultinomialNB())
(0.7528571428571429, AdaBoostClassifier())


In [7]:
maximo = max(resultados)
vencedor = resultados[maximo]

In [8]:
print("Vencerdor: ")
print(vencedor)

Vencerdor: 
OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0))


In [9]:
vencedor.fit(treino_dados, treino_marcacoes)

OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0))

In [10]:
teste_real(vencedor, validacao_dados, validacao_marcacoes)

Taxa de acerto do vencedor entre os algoritmos no mundo real: 100.0


In [11]:
acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

Taxa de acerto base: 82.608696


In [12]:
total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Total de teste: 23


# -----------------------------------------Criando Dicionário e Classificando Textos-----------------------------------------

    * Como classificar texto
    * Como implementar o código que cria um dicionário com todas as palavras distintas de um CSV
    * Como representar as palavras em números
    * Como representar uma frase em um array
    * Como utilizar algoritmo One vs Rest e realizar o teste de validação

Classificar textos em:
* Comercial : "Se eu comprar cinco anos antecipados, eu ganho algum desconto?"
* Técnico   : "O exercício 15 do curso de Java 1 está com a resposta errada. Pode conferir pf?"
* Carreira  : "Existe algum curso para cuidar do marketing da minha empresa?"

In [37]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.model_selection import cross_val_score

In [38]:
import nltk
#nltk.download('stopwords')
#nltk.download('rslp') # "removedor de sufixo da lingua portuguesa"
#nltk.download('punkt') 

In [39]:
classificacoes = pd.read_csv('emails.csv')
textosPuros = classificacoes['email']
#Recuperando textos em tokens e em minúsculo
textosQuebrados = textosPuros.str.lower().str.split(' ')
dicionario = set()

In [40]:
for lista in textosQuebrados:
    dicionario.update(lista)

In [41]:
#dicionario

In [42]:
totalDePalavras = len(dicionario)
tuplas = zip(dicionario, range(totalDePalavras))
tradutor = {palavra: indice for palavra, indice in tuplas}
print(totalDePalavras)

364


In [43]:
tradutor

{'': 0,
 'programação.': 1,
 'interesse': 2,
 'erro': 3,
 'devo': 4,
 'interessado': 5,
 'plano,': 6,
 'aplicações': 7,
 'dependente': 8,
 'conta': 9,
 'olá,': 10,
 'versão': 11,
 'de': 12,
 'nome': 13,
 'aulas,': 14,
 'começar,': 15,
 'filho?': 16,
 'almejando': 17,
 'necessário': 18,
 'programar,': 19,
 'comunidade': 20,
 'plano': 21,
 'controle': 22,
 'conhecimento': 23,
 'sobre': 24,
 'faço?': 25,
 'serei': 26,
 'sistemas': 27,
 'google': 28,
 'ficamos': 29,
 'paypal,': 30,
 'quem': 31,
 'cancelou': 32,
 'próprio': 33,
 'coloquei': 34,
 'não': 35,
 'qual': 36,
 'trabalhar': 37,
 'sei': 38,
 'e': 39,
 'indicada': 40,
 'faculdade': 41,
 'algumas': 42,
 'trocar.': 43,
 'programação,': 44,
 'desejo-te': 45,
 'identifiquei': 46,
 'desse': 47,
 'pai': 48,
 'diante': 49,
 '3': 50,
 'paypal': 51,
 'vista': 52,
 'ajudar': 53,
 'meus': 54,
 'que': 55,
 'equipe': 56,
 'django': 57,
 'distancia.': 58,
 'inicialmente.': 59,
 'ontem': 60,
 'carreira': 61,
 'necessidade': 62,
 'posso': 63,
 'trav

* Logo, na hora de vetorizar o tezto, devo pegar as raízes das palavras contidas nele, já que o tradutor contém as raízes

In [44]:
def vetorizar_texto(texto, tradutor):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if palavra in tradutor:
            posicao = tradutor[palavra]
            vetor[posicao] += 1

    return vetor

* Vetorizando todos os textos do meu arquivo 

In [31]:
vetoresDeTexto = [vetorizar_texto(texto, tradutor) for texto in textosQuebrados]

In [32]:
marcas = classificacoes['classificacao']

X = vetoresDeTexto
Y = marcas.tolist()

In [33]:
porcentagem_de_treino = 0.8

tamanho_de_treino = int(porcentagem_de_treino * len(Y))
tamanho_de_validacao = len(Y) - tamanho_de_treino

print(tamanho_de_treino)

34


In [34]:
treino_dados = X[0:tamanho_de_treino]
treino_marcacoes = Y[0:tamanho_de_treino]

validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

In [35]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv=k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print(msg)
    return taxa_de_acerto

In [36]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

resultados = {}

modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state=0))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

Taxa de acerto do OneVsRest: 0.7166666666666667


# -----------------------------------------Intro Pré Processamento-----------------------------------------

    * Stop words
    * Como remover as stop words do dicionário
    * Como instalar o nltk e seus módulos
    * Como extrair as raízes das palavras
    * Como separar as palavras de uma frase com o tokenize
    * Como remover palavras com pontuações do dicionário

In [46]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.model_selection import cross_val_score

In [49]:
import nltk
#nltk.download('stopwords')
#nltk.download('rslp') # "removedor de sufixo da lingua portuguesa"
#nltk.download('punkt') 

Auxílios do nltk usados:
    * stopwords
    * stemmer
    * wordTokenizer

In [50]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer() 


#frase = "bora ali compra um pão? bora bora!"
#x = tokenizer = nltk.tokenize.word_tokenize(frase)
#print(x)

In [51]:
classificacoes = pd.read_csv('emails.csv')
textosPuros = classificacoes['email']
#Recuperando textos em tokens e em minúsculo
frases = textosPuros.str.lower()
textosQuebrados = [nltk.tokenize.word_tokenize(frase) for frase in frases]

In [52]:
#dicionario

* Utilizando o stemmer para recupear raízes das palavras e colocar no dicionário

In [53]:
dicionario = set()
for lista in textosQuebrados:
    validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra) > 2]
    dicionario.update(validas)

In [55]:
totalDePalavras = len(dicionario)
print(f'total de palavras: {totalDePalavras}')
print(dicionario)
tuplas = zip(dicionario, range(totalDePalavras))
tradutor = {palavra: indice for palavra, indice in tuplas}

total de palavras: 230
{'imprim', 'bas', 'dia', 'digit', 'coloq', 'prát', 'capaz', 'pra', 'aplic', 'própri', 'depend', 'pass', 'antig', 'tod', 'di', 'hav', 'complet', 'plan', 'algum', 'post', 'comput', 'pert', 'ach', 'poss', 'vou', 'seguint', 'gráf', 'qu', 'mim', 'ont', 'premiun', 'muit', 'áre', 'outr', 'web', 'dev', 'pod', 'realiz', 'empr', 'comunidad', 'desenvolv', 'emit', 'certific', 'vers', 'alguém', 'expir', 'naveg', 'filh', 'sit', 'program', 'pal', 'trabalh', 'faç', 'vontad', 'alur', 'form', 'começ', 'gost', 'vist', 'vez', 'boa', 'olá', 'por', 'aqu', 'premium', 'bancár', 'fic', 'independ', 'possu', 'cinc', 'ótim', 'interfac', 'faz', 'bom', 'ganh', 'antecipad', 'io', 'dess', 'part', 'colun', 'ambi', 'segund', 'duvid', 'dúv', 'explic', 'depo', 'opt', 'sab', 'fórum', 'us', 'plataform', 'cri', 'pouc', 'desejo-t', 'err', 'quer', 'torn', 'sent', 'qual', 'window', 'trav', 'minut', 'desd', 'laravel', 'melhor', 'cadastr', 'faculdad', 'sei', 'troc', 'entend', 'cont', 'víde', 'mac', 'escolh

In [56]:
tradutor

{'imprim': 0,
 'bas': 1,
 'dia': 2,
 'digit': 3,
 'coloq': 4,
 'prát': 5,
 'capaz': 6,
 'pra': 7,
 'aplic': 8,
 'própri': 9,
 'depend': 10,
 'pass': 11,
 'antig': 12,
 'tod': 13,
 'di': 14,
 'hav': 15,
 'complet': 16,
 'plan': 17,
 'algum': 18,
 'post': 19,
 'comput': 20,
 'pert': 21,
 'ach': 22,
 'poss': 23,
 'vou': 24,
 'seguint': 25,
 'gráf': 26,
 'qu': 27,
 'mim': 28,
 'ont': 29,
 'premiun': 30,
 'muit': 31,
 'áre': 32,
 'outr': 33,
 'web': 34,
 'dev': 35,
 'pod': 36,
 'realiz': 37,
 'empr': 38,
 'comunidad': 39,
 'desenvolv': 40,
 'emit': 41,
 'certific': 42,
 'vers': 43,
 'alguém': 44,
 'expir': 45,
 'naveg': 46,
 'filh': 47,
 'sit': 48,
 'program': 49,
 'pal': 50,
 'trabalh': 51,
 'faç': 52,
 'vontad': 53,
 'alur': 54,
 'form': 55,
 'começ': 56,
 'gost': 57,
 'vist': 58,
 'vez': 59,
 'boa': 60,
 'olá': 61,
 'por': 62,
 'aqu': 63,
 'premium': 64,
 'bancár': 65,
 'fic': 66,
 'independ': 67,
 'possu': 68,
 'cinc': 69,
 'ótim': 70,
 'interfac': 71,
 'faz': 72,
 'bom': 73,
 'ganh': 7

* Logo, na hora de vetorizar o tezto, devo pegar as raízes das palavras contidas nele, já que o tradutor contém as raízes

In [57]:
def vetorizar_texto(texto, tradutor, stemmer):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1
    return vetor

In [58]:
vetoresDeTexto = [vetorizar_texto(texto, tradutor, stemmer) for texto in textosQuebrados]

In [59]:
marcas = classificacoes['classificacao']

X = vetoresDeTexto
Y = marcas

In [60]:
porcentagem_de_treino = 0.8

tamanho_de_treino = int(porcentagem_de_treino * len(Y))
tamanho_de_validacao = len(Y) - tamanho_de_treino

print(tamanho_de_treino)

34


In [61]:
treino_dados = X[0:tamanho_de_treino]
treino_marcacoes = Y[0:tamanho_de_treino]

validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

In [62]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv=k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print(msg)
    return taxa_de_acerto

In [63]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [64]:
resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state=0))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest


from sklearn.multiclass import OneVsOneClassifier

modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state=0))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB

modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier

modeloAdaBoost = AdaBoostClassifier(random_state=0)
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost


print(resultados)

maximo = max(resultados)
vencedor = resultados[maximo]

print("Vencerdor: ")
print(vencedor)

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Taxa de acerto do OneVsRest: 0.7416666666666666
Taxa de acerto do OneVsOne: 0.7416666666666667
Taxa de acerto do MultinomialNB: 0.8
Taxa de acerto do AdaBoostClassifier: 0.6166666666666666
{0.7416666666666666: OneVsRestClassifier(estimator=LinearSVC(random_state=0)), 0.7416666666666667: OneVsOneClassifier(estimator=LinearSVC(random_state=0)), 0.8: MultinomialNB(), 0.6166666666666666: AdaBoostClassifier(random_state=0)}
Vencerdor: 
MultinomialNB()
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 77.77777777777777
Taxa de acerto base: 44.444444
Total de teste: 9
